In [27]:
import pandas as pd
import numpy as np
import os

def filepath(f):
    d = os.path.join(os.getcwd(), 'data', 'fraud_detection_data', f)
    return d    

#Import training datasets
train_label=pd.read_csv(filepath("Train.csv"))
train_beneficiary=pd.read_csv(filepath("Train_Beneficiarydata.csv"))
train_inpatient=pd.read_csv(filepath("Train_Inpatientdata.csv"))
train_outpatient=pd.read_csv(filepath("Train_Outpatientdata.csv"))

In [28]:
train_label = train_label.replace({"PotentialFraud": "No"}, 0)
train_label = train_label.replace({"PotentialFraud": "Yes"}, 1)

In [29]:
##Replacing 2 with 0 for chronic conditions ,that means chroniv condition No is 0 and yes is 1
train_beneficiary = train_beneficiary.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                           'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2, 
                           'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2, 
                           'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2 }, 0)

train_beneficiary = train_beneficiary.replace({'RenalDiseaseIndicator': 'Y'}, 1)

In [30]:
# Combine training datasets 
train_inpatient['inpatient'] = 1
train_outpatient['inpatient'] = 0
train=pd.merge(train_outpatient,train_inpatient,
                              left_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode','inpatient'],
                              right_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode','inpatient']
                              ,how='outer')

train_alldata=pd.merge(train,train_beneficiary,left_on='BeneID',right_on='BeneID',how='inner')

C:\Users\aengu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\reshape\merge.py:1213: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


In [31]:
# Drop constant columns
for col in train_alldata.columns:
    if train_alldata[col].nunique() <= 1:
        train_alldata.drop([col], inplace=True, axis=1)
        print(col, "dropped as it is a constant.")

ClmProcedureCode_6 dropped as it is a constant.


In [32]:
#Merging with Provider
train_allprovider=pd.merge(train_label,train_alldata,on='Provider')

In [33]:
# Process datetime columns -- Get age of claimant
train_allprovider["DOD"] = pd.to_datetime(train_allprovider["DOD"])
train_allprovider["DOB"] = pd.to_datetime(train_allprovider["DOB"])

latest_date = pd.to_datetime(train_allprovider["ClaimEndDt"]).max()
latest_date


Timestamp('2009-12-31 00:00:00')

In [34]:
# Filling the NA values in DOD column of alive claimants with latest claim date
train_allprovider["DOD"] = train_allprovider["DOD"].fillna(latest_date)
train_allprovider["age"] = (train_allprovider["DOD"] - train_allprovider["DOB"])/np.timedelta64(1,'Y')

In [35]:
# Get claim duration column and remove datetime columns of claim start and end
train_allprovider["claim_duration"] = (pd.to_datetime(train_allprovider["ClaimEndDt"]) - pd.to_datetime(train_allprovider["ClaimStartDt"]))/np.timedelta64(1,'D')
train_allprovider["claim_duration"] = train_allprovider['claim_duration'].astype('float64')

In [36]:
# Handle time spent in hospital
train_allprovider["time_under_care"] = (pd.to_datetime(train_allprovider["DischargeDt"]) - pd.to_datetime(train_allprovider["AdmissionDt"]))/np.timedelta64(1,'D')
train_allprovider["time_under_care"] = train_allprovider['time_under_care'].astype('float64')
train_allprovider["time_under_care"] = train_allprovider["time_under_care"].fillna(0.0)

In [37]:
#Removing unecessary columns
drop_cols = [
    'NoOfMonths_PartACov', 
    'NoOfMonths_PartBCov', 
    'State', 
    'County', 
    'AttendingPhysician',
    'OperatingPhysician', 
    'OtherPhysician', 
    'DiagnosisGroupCode',
    'DOB',
    'DOD',
    'ClaimStartDt',
    'ClaimEndDt',
    'BeneID',
    'AdmissionDt',
    'DischargeDt'
    ]

train_allprovider.drop(drop_cols, axis=1, inplace=True)

In [38]:
#Read in icd9 data
diagnosis=pd.read_csv("data/medical_codes_data/icd9_diagnosis.csv", dtype={'code': str})

procedures=pd.read_csv("data/medical_codes_data/icd9_procedure.csv", dtype={'code': str})

In [39]:
diagnosis_dict = {}
unique_diagnosis = diagnosis['description_1'].unique()
for (i,row) in diagnosis.iterrows():
    diagnosis_dict[row["code"]] = row["description_1"]

procedures_dict = {}
unique_procedures = procedures['description_1'].unique()
for (i,row) in procedures.iterrows():
    procedures_dict[row["code"]] = row["description_1"]


In [40]:
unique_procedures

array(['Procedures And Interventions , Not Elsewhere Classified',
       'Operations On The Nervous System',
       'Operations On The Endocrine System', 'Operations On The Eye',
       'Other Miscellaneous Diagnostic And Therapeutic Procedures',
       'Operations On The Ear',
       'Operations On The Nose, Mouth, And Pharynx',
       'Operations On The Respiratory System',
       'Operations On The Cardiovascular System',
       'Operations On The Hemic And Lymphatic System',
       'Operations On The Digestive System',
       'Operations On The Urinary System',
       'Operations On The Male Genital Organs',
       'Operations On The Female Genital Organs',
       'Obstetrical Procedures',
       'Operations On The Musculoskeletal System',
       'Operations On The Integumentary System',
       'Miscellaneous Diagnostic And Therapeutic Procedures'],
      dtype=object)

In [41]:
diagnoses_code_in_table = {}
for i,d in enumerate(unique_diagnosis):
    diagnoses_code_in_table[d] = i+1

procedures_code_in_table = {}
for i,p in enumerate(unique_procedures):
    procedures_code_in_table[p] = i+1



In [42]:
# Create a list of columns to process
columns_to_process = ['ClmDiagnosisCode_' + str(i) for i in range(1, 11)]
not_present_set_diagnosis = set()

# Replace the codes with the descriptions
diagnosis_descriptor = []
for (i,row) in train_allprovider.iterrows():
    descriptors = set()
    for col in columns_to_process:
        adm = False
        try:
            diag_num = str(diagnosis_dict[row[col]])
            new_code2 = diagnoses_code_in_table[diag_num]
        except:
            if pd.isnull(row[col]):
                continue
            else:
                set.add(not_present_set_diagnosis, row[col])
                continue
        descriptors.add(new_code2)
    diagnosis_descriptor.append(descriptors)



In [43]:
adm_diag = []
for (i,row) in train_allprovider.iterrows():
    ind = False
    value = row["ClmAdmitDiagnosisCode"]
    for col in columns_to_process:
        if ind == True:
            continue
        if pd.isnull(value):
            continue
        if str(value) == str(row[col]):
            adm_diag.append(1)
            ind = True
    if ind == False:
        adm_diag.append(0)
len(adm_diag)

train_allprovider["admitDiagInFinalDiagnosis"] = adm_diag

In [44]:
not_present_set_diagnosis

{'0430', '0431', '0449', '71860'}

In [45]:
columns_to_process2 = ['ClmProcedureCode_' + str(i) for i in range(1, 6)]
admit_diag = []

not_present_set_procedure = set()
procedures_descriptor = []
for (i,row) in train_allprovider.iterrows():
    procedures = set()
    for col in columns_to_process2:
        try:
            n = str(int(row[col]))
            new_code = procedures_dict[n]
            new_code2 = procedures_code_in_table[new_code]
        except:
            if pd.isnull(row[col]):
                continue
            else:
                set.add(not_present_set_procedure, str(int(row[col])))
                continue
        procedures.add(new_code2)
    procedures_descriptor.append(procedures)

In [46]:
not_present_set_procedure

{'239', '296', '309', '407'}

In [47]:
diagnoses_columns = {}

for j in [i for i in diagnoses_code_in_table.values()]:
    diagnoses_columns[j] = []

for d in diagnosis_descriptor:
    for k,v in diagnoses_columns.items():
        if k in d:
            v.append(1)
        else:
            v.append(0)

procedures_columns = {}

for j in [i for i in procedures_code_in_table.values()]:
    procedures_columns[j] = []

for p in procedures_descriptor:
    for k,v in procedures_columns.items():
        if k in p:
            v.append(1)
        else:
            v.append(0)
    

In [48]:
final_diagnosis_cols = {}
for k, name in zip(diagnoses_columns.keys(), diagnoses_code_in_table.values()):
    new_name = "diagnosis_"+str(name)
    final_diagnosis_cols[new_name] = diagnoses_columns[k]

final_procedures_cols = {}
for k, name in zip(procedures_columns.keys(), procedures_code_in_table.values()):
    new_name = "procedure_"+str(name)
    final_procedures_cols[new_name] = procedures_columns[k]


In [49]:
train_allprovider.drop(
    columns_to_process + columns_to_process2,
    axis = 1,
    inplace = True
)
train_allprovider.drop(
    ["ClmAdmitDiagnosisCode"],
    axis = 1,
    inplace = True
)

train_allprovider.head()

,Provider,PotentialFraud,ClaimID,InscClaimAmtReimbursed,DeductibleAmtPaid,inpatient,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,...,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,age,claim_duration,time_under_care,admitDiagInFinalDiagnosis
0,PRV51001,0,CLM733300,20,0.0,0,1,1,1,1,...,0,0,0,0,300,110,79.670356,0.0,0.0,0
1,PRV51001,0,CLM372475,700,0.0,0,1,1,0,1,...,0,1,0,0,720,10,66.837786,1.0,0.0,0
2,PRV51001,0,CLM748221,900,0.0,0,2,1,0,0,...,1,0,0,0,1380,370,76.417722,0.0,0.0,0
3,PRV51001,0,CLM272936,500,0.0,0,1,1,0,1,...,0,0,2020,1068,6700,2700,73.835876,1.0,0.0,0
4,PRV51001,0,CLM58316,36000,1068.0,1,1,1,0,0,...,0,1,36000,1068,3520,140,69.501769,4.0,4.0,0


In [50]:
final_train_df = pd.concat(
    [
        pd.concat(
            [
                train_allprovider,
                pd.DataFrame(
                    final_diagnosis_cols
                )
            ],
            axis = 1
        ),
        pd.DataFrame(
            final_procedures_cols
        )
    ], 
    axis = 1
)

In [52]:
final_train_df.to_csv("processed_data/final_training_set.csv", index=False)